In [1]:
from hexathello import aiPlayers, autoPlayer, engine, history, jable, printing

import numpy as np

from os import path

# -- Settings
game_size: int = 5
player_count: int = 2

In [2]:
# We want to get some baseline data for learning, using a Greendom Agent, which is smarter
#   than a simple random agent
# We store this at `data/histories/greendom_size-5_players-2.json`
histories_dir: str = path.join(
    'data',
    'history',
    'examples'
)
assert path.isdir(
    histories_dir
)

baseline_data_path: str = path.join(
    histories_dir,
    'greendom_size-{}_players-{}.json'.format(
        game_size, player_count
    )
)
baseline_data: jable.JyFrame

max_data: int = 20000
existing_data: int = 0
if path.isfile( baseline_data_path ):
    print("# Reading data from {}".format( baseline_data_path ))
    baseline_data: jable.JyFrame = jable.read_file(
        baseline_data_path
    )
    existing_data = len( baseline_data )
#
else:
    print("# Starting new data at {}".format( baseline_data_path ))
    baseline_data: jable.JyFrame = history.new_literalHistory(
        player_count = player_count,
        size = game_size
    )
    baseline_data = history.history_asInt( baseline_data )
    
    # We need to change scores and winner to be shift
    baseline_data.makeColumn_shift( "winner" )
    baseline_data.makeColumn_shift( "scores" )
#/if path.isfile( baseline_data_path )/else

ai_0: aiPlayers.GreedyHexAgent = aiPlayers.GreedyHexAgent(
    size = game_size,
    player_count = player_count,
    player_id = 0,
    p_random = 0.4
)

ai_1: aiPlayers.GreedyHexAgent = aiPlayers.GreedyHexAgent(
    size = game_size,
    player_count = player_count,
    player_id = 1,
    p_random = 0.4
)
    
play_index: int = 0
if existing_data >= max_data:
    print("# Already have existing_data={}".format( existing_data) )
#

while existing_data < max_data:
    print("# Starting play_index={}".format(play_index))
    literalHistory: jable.JyFrame = autoPlayer.runHexathello_withAgents(
        agents = [ai_0, ai_1],
        size = game_size,
        logging_level = 0
    )
    
    baseline_data.extend(
        history.history_asInt(
            literalHistory
        )
    )
    
    baseline_data.write_file(
        fp = baseline_data_path
    )
    
    existing_data = len( baseline_data )
    play_index += 1
    
    # Safety Valve
    if play_index > max_data:
        raise Exception("Too large play_index={}".format(play_index))
    #
#/while existing_data < max_data

printing.prettyprint( baseline_data[:50] )

# Starting new data at data/history/examples/greendom_size-5_players-2.json
# Starting play_index=0
Early end, no move for any player
RESULT: 20 - 22; Player 1 wins
# Game done
{'winner': 1, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 1, 'scores': [20, 22]}
# appending with strict=False
# Starting play_index=1
Early end, no move for any player
RESULT: 21 - 21; Tie among 0, 1
# Game done
{'winner': None, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 1, 'scores': [21, 21]}
# appending with strict=False
# Starting play_index=2
Early end, no move for any player
RESULT: 14 - 28; Player 1 wins
# Game done
{'winner': 1, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 1, 'scores': [14, 28]}
# appending with strict=False
# Starting play_index=3
Early end, no move for any player
RESULT: 19 - 23; Player 1 wins
# G

Early end, no move for any player
RESULT: 17 - 25; Player 1 wins
# Game done
{'winner': 1, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 0, 'scores': [17, 25]}
# appending with strict=False
# Starting play_index=32
Early end, no move for any player
RESULT: 12 - 30; Player 1 wins
# Game done
{'winner': 1, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 1, 'scores': [12, 30]}
# appending with strict=False
# Starting play_index=33
Early end, no move for any player
RESULT: 33 - 9; Player 0 wins
# Game done
{'winner': 0, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 0, 'scores': [33, 9]}
# appending with strict=False
# Starting play_index=34
Early end, no move for any player
RESULT: 18 - 16; Player 0 wins
# Game done
{'winner': 0, 'turn_index': 28, 'size': 5, 'game_complete': True, 'empty_count': 27, 'player_c

# Starting play_index=65
Early end, no move for any player
RESULT: 9 - 31; Player 1 wins
# Game done
{'winner': 1, 'turn_index': 34, 'size': 5, 'game_complete': True, 'empty_count': 21, 'player_count': 2, 'current_player': 1, 'scores': [9, 31]}
# appending with strict=False
# Starting play_index=66
Early end, no move for any player
RESULT: 27 - 15; Player 0 wins
# Game done
{'winner': 0, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 0, 'scores': [27, 15]}
# appending with strict=False
# Starting play_index=67
Early end, no move for any player
RESULT: 20 - 22; Player 1 wins
# Game done
{'winner': 1, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 1, 'scores': [20, 22]}
# appending with strict=False
# Starting play_index=68
Early end, no move for any player
RESULT: 29 - 13; Player 0 wins
# Game done
{'winner': 0, 'turn_index': 36, 'size': 5, 'game_complete': True, 'em

# Starting play_index=98
Early end, no move for any player
RESULT: 23 - 19; Player 0 wins
# Game done
{'winner': 0, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 1, 'scores': [23, 19]}
# appending with strict=False
# Starting play_index=99
Early end, no move for any player
RESULT: 20 - 22; Player 1 wins
# Game done
{'winner': 1, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 0, 'scores': [20, 22]}
# appending with strict=False
# Starting play_index=100
Early end, no move for any player
RESULT: 16 - 26; Player 1 wins
# Game done
{'winner': 1, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 0, 'scores': [16, 26]}
# appending with strict=False
# Starting play_index=101
Early end, no move for any player
RESULT: 27 - 15; Player 0 wins
# Game done
{'winner': 0, 'turn_index': 36, 'size': 5, 'game_complete': True,

# Starting play_index=131
Early end, no move for any player
RESULT: 22 - 20; Player 0 wins
# Game done
{'winner': 0, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 0, 'scores': [22, 20]}
# appending with strict=False
# Starting play_index=132
Early end, no move for any player
RESULT: 24 - 18; Player 0 wins
# Game done
{'winner': 0, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 0, 'scores': [24, 18]}
# appending with strict=False
# Starting play_index=133
Early end, no move for any player
RESULT: 16 - 26; Player 1 wins
# Game done
{'winner': 1, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 1, 'scores': [16, 26]}
# appending with strict=False
# Starting play_index=134
Early end, no move for any player
RESULT: 12 - 29; Player 1 wins
# Game done
{'winner': 1, 'turn_index': 35, 'size': 5, 'game_complete': Tru

...

# Starting play_index=438
Early end, no move for any player
RESULT: 13 - 23; Player 1 wins
# Game done
{'winner': 1, 'turn_index': 30, 'size': 5, 'game_complete': True, 'empty_count': 25, 'player_count': 2, 'current_player': 1, 'scores': [13, 23]}
# appending with strict=False
# Starting play_index=439
Early end, no move for any player
RESULT: 8 - 34; Player 1 wins
# Game done
{'winner': 1, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 1, 'scores': [8, 34]}
# appending with strict=False
# Starting play_index=440
Early end, no move for any player
RESULT: 21 - 21; Tie among 0, 1
# Game done
{'winner': None, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 0, 'scores': [21, 21]}
# appending with strict=False
# Starting play_index=441
Early end, no move for any player
RESULT: 32 - 10; Player 0 wins
# Game done
{'winner': 0, 'turn_index': 36, 'size': 5, 'game_complete': T

# Starting play_index=468
Early end, no move for any player
RESULT: 15 - 27; Player 1 wins
# Game done
{'winner': 1, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 1, 'scores': [15, 27]}
# appending with strict=False
# Starting play_index=469
Early end, no move for any player
RESULT: 22 - 20; Player 0 wins
# Game done
{'winner': 0, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 1, 'scores': [22, 20]}
# appending with strict=False
# Starting play_index=470
Early end, no move for any player
RESULT: 14 - 28; Player 1 wins
# Game done
{'winner': 1, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 1, 'scores': [14, 28]}
# appending with strict=False
# Starting play_index=471
Early end, no move for any player
RESULT: 22 - 20; Player 0 wins
# Game done
{'winner': 0, 'turn_index': 36, 'size': 5, 'game_complete': Tru

# Starting play_index=498
Early end, no move for any player
RESULT: 22 - 20; Player 0 wins
# Game done
{'winner': 0, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 0, 'scores': [22, 20]}
# appending with strict=False
# Starting play_index=499
Early end, no move for any player
RESULT: 26 - 16; Player 0 wins
# Game done
{'winner': 0, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 0, 'scores': [26, 16]}
# appending with strict=False
# Starting play_index=500
Early end, no move for any player
RESULT: 20 - 22; Player 1 wins
# Game done
{'winner': 1, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 1, 'scores': [20, 22]}
# appending with strict=False
# Starting play_index=501
Early end, no move for any player
RESULT: 18 - 24; Player 1 wins
# Game done
{'winner': 1, 'turn_index': 36, 'size': 5, 'game_complete': Tru

# Starting play_index=528
Early end, no move for any player
RESULT: 28 - 14; Player 0 wins
# Game done
{'winner': 0, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 1, 'scores': [28, 14]}
# appending with strict=False
# Starting play_index=529
Early end, no move for any player
RESULT: 33 - 9; Player 0 wins
# Game done
{'winner': 0, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 1, 'scores': [33, 9]}
# appending with strict=False
# Starting play_index=530
Early end, no move for any player
RESULT: 26 - 16; Player 0 wins
# Game done
{'winner': 0, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 0, 'scores': [26, 16]}
# appending with strict=False
# Starting play_index=531
Early end, no move for any player
RESULT: 30 - 12; Player 0 wins
# Game done
{'winner': 0, 'turn_index': 36, 'size': 5, 'game_complete': True,

# Starting play_index=558
Early end, no move for any player
RESULT: 13 - 29; Player 1 wins
# Game done
{'winner': 1, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 1, 'scores': [13, 29]}
# appending with strict=False
# Starting play_index=559
Early end, no move for any player
RESULT: 23 - 19; Player 0 wins
# Game done
{'winner': 0, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 0, 'scores': [23, 19]}
# appending with strict=False
# Starting play_index=560
Early end, no move for any player
RESULT: 21 - 21; Tie among 0, 1
# Game done
{'winner': None, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 1, 'scores': [21, 21]}
# appending with strict=False
# Starting play_index=561
Early end, no move for any player
RESULT: 26 - 16; Player 0 wins
# Game done
{'winner': 0, 'turn_index': 36, 'size': 5, 'game_complete':

In [3]:
# In `quickstart_training` we'll use data we created to train a smarter AI